# Description

This notebook generates OTUs at 97% and assigns taxonomy. Note that OTUs are generated from the whole dataset.

* Creating OTUs with usearch 
* Taxonomy assignment with QIIME using SILVA database version 132.
* Filtering out Chloroplasts and Mitochondria with Mothur.

# Setting variables

In [1]:
import os

workDir = '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning'
qcDirs = ['/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/160506_FC2Grad_1',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/160713_FC2Unfrac',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/AS_Pool1_redo',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/Chantal_Pool9',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/final_Enr_soilHealthLib',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib14',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib15',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib16',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib2',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib3',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib4',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib5',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib6',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib7',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib8',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib9',
          '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/Spencer_unfrac_redo']
          
dbDir = '/home/sam/databases/SILVA_132_QIIME_release'
dbSeqs = os.path.join(dbDir, 'rep_set/rep_set_16S_only/97/silva_132_97_16S.fna')
dbTax = os.path.join(dbDir, 'taxonomy/16S_only/97/consensus_taxonomy_7_levels.txt')

nprocs = 20

# Init

In [2]:
import os
import sys
import glob
import pandas as pd
#from cogent.parse.fasta import MinimalFastaParser as parse
from Bio import SeqIO
%load_ext rpy2.ipython


In [3]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘gridExtra’


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:dplyr’:

    combine


  warnings.warn(x, RRuntimeWarning)


In [4]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)
%cd $workDir

/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning


### Combinding final QC files from each library

In [15]:
qcFiles = [os.path.join(x, 'QC', 'finalQC.fasta') for x in qcDirs]
# check to make sure files exist
for F in qcFiles:
    if not os.path.isfile(F):
        print ('Cannot find: {}'.format(F))
    else:
        print ('Found file: {}'.format(F))
        file_size = os.path.getsize(F) / 1000000
        print ('  File size: {} Mb'.format(file_size))
        nseq = !grep -c '>' $F
        print ('  Number of sequences: {}'.format(nseq[0]))

Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/160506_FC2Grad_1/QC/finalQC.fasta
  File size: 466.076111 Mb
  Number of sequences: 1610005
Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/160713_FC2Unfrac/QC/finalQC.fasta
  File size: 1315.935248 Mb
  Number of sequences: 4658425
Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/AS_Pool1_redo/QC/finalQC.fasta
  File size: 163.918342 Mb
  Number of sequences: 569787
Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/Chantal_Pool9/QC/finalQC.fasta
  File size: 248.705146 Mb
  Number of sequences: 907755
Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/final_Enr_soilHealthLib/QC/finalQC.fasta
  File size: 127.227247 Mb
  Number of sequences: 444962
Found file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib14/QC/finalQC.fasta
  File size: 4171.885164 Mb
 

In [18]:
# cat command
qcFinal = os.path.join(workDir, 'finalQC.fasta')
cmd = ' '.join(['cat'] + qcFiles + ['> {}'.format(qcFinal)])
!$cmd
 
# file stats
file_size = os.path.getsize(qcFinal) / 1000000
print ('File size: {} Mb'.format(file_size))
nseq = !grep -c '>' $qcFinal
print ('Number of sequences: {}'.format(nseq[0]))

File size: 20680.025109 Mb
Number of sequences: 71891874


# Just unique sequences

In [19]:
cmd =  'mothur "#unique.seqs(fasta={})"'.format(qcFinal)
!$cmd | tail -n 30

71870000	31208219
71871000	31209145
71872000	31210026
71873000	31210394
71874000	31210917
71875000	31211460
71876000	31212062
71877000	31212691
71878000	31213346
71879000	31214000
71880000	31214658
71881000	31215309
71882000	31215941
71883000	31216574
71884000	31217265
71885000	31217952
71886000	31218686
71887000	31219445
71888000	31220174
71889000	31220849
71890000	31221589
71891000	31222281
71891874	31222885

Output File Names: 
/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC.names
/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC.unique.fasta


mothur > quit()


In [20]:
qcFinalUniq = os.path.splitext(qcFinal)[0] + '.unique.fasta'

ret = !grep -c ">" $qcFinalUniq
print ('Number of unique sequences: {}'.format(ret[0]))

Number of unique sequences: 31222885


# Formatting seq names for usearch

In [21]:
qcFinalName = os.path.splitext(qcFinal)[0] + '.names'

counts = {}
with open(qcFinalName) as iFH:
    for line in iFH:
        seedID, seqIDs = line.split("\t")
        count = len(seqIDs.split(","))
        counts[seedID] = count 

In [22]:
qcFinalUs = os.path.splitext(qcFinalUniq)[0] + '.usearch_names.fasta'

with open(qcFinalUs, 'w') as oFH:
    for entry in SeqIO.parse(open(qcFinalUniq),'fasta'):
        n, s = entry.id, entry.seq.tostring()
        if counts[n] > 1:
            oFH.write(">%s;size=%s;\n%s\n"%(n,counts[n],s))
        else:
            continue

/home/sam/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:345: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [23]:
#!cd $workDir; \
!head -n 6 $qcFinalUs

>MR.F.13C-Ami.D3.R2_Frac10_14;size=326928;
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTCCGCTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTTGTGACTGGCGGGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGG
>MR.F.13C-Ami.D3.R2_Frac28_21;size=15444;
TACGGAGGGTGCAAGCGTTATCCGGATTCACTGGGTTTAAAGGGTGCGTAGGCGGGTTGGTAAGTCCGTGGTGAAATCTCCAAGCTTAACTTGGAAACTGCCGTGGATACTATCAATCTTGAATATCGTGGAGGTGAGCGGAATATGTCATGTAGCGGTGAAATGCTTAGATATGACATAGAACACCCATTGCGAAGGCAGCTCGCTACACGGTTATTGACGCTGAGGCACGAAAGCGTGGGGATCAAACAGG
>MR.F.13C-Ami.D1.R1_Frac15_28;size=2240;
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGCTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTTGTGACTGGCGGGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGG


# Usearch pipeline

In [24]:
qcFinalUsSort = os.path.splitext(qcFinalUs)[0] + '_sorted.fasta'
cmd = 'usearch -sortbysize {} -fastaout {} -minsize 2'.format(qcFinalUs, qcFinalUsSort)
!$cmd 

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:09 1.3Gb   100.0% Reading /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC.unique.usearch_names.fasta
00:09 1.2Gb  Getting sizes                                                                                                          
00:10 1.3Gb  Sorting 3853318 sequences
00:22 1.3Gb   100.0% Writing output


In [ ]:
!head $qcFinalUsSort
!echo 
!tail $qcFinalUsSort

>MR.A.13C-Ami.D6.R1_678;size=634457;
TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGACCGATGTGAAATCCC
CGGGCTCAACCTGGGAACTGCATTGGTGACTGGCAAGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTG
AAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTG
GGGAGCAAACAGG
>MR.F.13C-Ami.D3.R2_Frac19_1092930;size=344432;
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGACCGATGTGAAATCCC
CGGGCTCAACCTGGGAACTGCATTGGTGACTGGCAAGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTG
AAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTG
GGGAGCAAACAGG

>MF.A.151026.12C-Van.D02_14731676;size=2;
TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGTGTGCGCAGGTGGCCGCGCAAGTCGAGTGTGAAATCCC
CGAGCTTAACTTGGGAATTGCGCTCGAAACTACGTGGATCGAGTGTGGCAGAGGAAGGTGGAATTCCACGTGTAGCGGTG
AAATGCGTAGAGATGTGGAGGAACACCGATGGCGAAGGCAGCCTTCTGGGCCAACACTGACGCTCATGCACGAAAGCGTG
GGGAGCAAACAGG
>BH.F.151026.12C-Ami.D02_14731666;size=2;
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGTAGGCGGTTAT

In [ ]:
otuFile = 'otus.fasta'
cmd = 'usearch -cluster_otus {} -otus {}'.format(qcFinalUsSort, otuFile)
!$cmd 

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu



In [ ]:
otuFileRn = os.path.splitext(otuFile)[0] + 'n.fasta'

In [ ]:
%%bash -s "$otuFile" "$otuFileRn"

 bioawk -c fastx '{print ">" "OTU" "." NR "\n" $seq}' $1 > $2

In [53]:
!head -n 4 $otuFileRn

>OTU.1
TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGACCGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATTGGTGACTGGCAAGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGG
>OTU.2
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTCGCTAAGACAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTTGTGACTGGCGGGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGCCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGG


STOP

### Assigning taxonomy

In [ ]:
otuFileTax = os.path.splitext(otuFileRn)[0] + '_tax'
!assign_taxonomy.py -r $dbSeqs -t $dbTax -i $otuFileRn -o $otuFileTax

### Removing Chloroplast, Eukaryal and mitochodria sequences

In [ ]:
otuTax = os.path.join(otuFileTax, 'otusn_tax_assignments.txt')

to_rm = 'Chloroplast|Eukaryota|Unassigned|Mitochondria'
to_rm_file = 'to_remove_tax.accnos'

awk_cmd = '{print \$1}'
cmd = 'egrep "{}" {} | awk "{}" > {}'.format(to_rm, otuTax, awk_cmd, to_rm_file)
!$cmd

In [ ]:
ret = !wc -l $to_rm_file
print ('Number of lines: {}'.format(ret[0]))
!head $to_rm_file

In [ ]:
cmd = 'mothur "#remove.seqs(fasta={}, accnos={})"'.format(otuFileRn, to_rm_file)
!$cmd | tail -n 30

In [56]:
otuFilePick = os.path.splitext(otuFileRn)[0] + '.pick.fasta'

ret = !grep -c ">" $otuFileRn
print ('Pre-filter: number of sequences: {}'.format(ret[0]))

ret = !grep -c ">" $otuFilePick
print ('Post-filter: number of sequences: {}'.format(ret[0]))

Pre-filter: number of sequences: 35906
Post-filter: number of sequences: 30810


# Mapping reads

In [ ]:
cmd_perl = 's/^>(.+)(_[^_]+)\n\$/>\$1\$2\_\$.;barcodelabel=\$1\n/' 

seqFile = os.path.splitext(qcFinal)[0] + '_usearchfmt.fasta'

cmd = 'perl -pe "{}" {} > {}'.format(cmd_perl, qcFinal, seqFile)
!$cmd

In [ ]:
!head -n 6 $seqFile

In [57]:
#see how many Gb this file is.
!du -h $seqFile

23G	/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.fasta


In [ ]:
#spliting file because it is too big
!pyfasta split -n 10 $seqFile


In [60]:
g = os.path.join(workDir, 'finalQC_usearchfmt.*.fasta')
fileList = glob.glob(g)
fileList 

['/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.06.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.09.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.08.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.03.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.02.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.01.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.04.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.05.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.00.fasta',
 '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.0

In [61]:
# running usearch on each split file
for f in fileList:
    sys.stderr.write('Processing {}\n'.format(f))

    ff,_ = os.path.splitext(f)
    _,i = os.path.splitext(ff)
    uc = 'readmap{}.uc'.format(i.lstrip('.')) 

    !usearch \
        -usearch_global $f \
        -db $otuFilePick \
        -strand plus -id 0.97 \
        -uc $uc \
        -threads $nprocs

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu



Processing /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.06.fasta


00:00 49Mb    100.0% Reading otusn.pick.fasta
00:00 15Mb    100.0% Masking (fastnucleo)    
00:00 16Mb    100.0% Word stats          
00:00 16Mb    100.0% Alloc rows
00:01 45Mb    100.0% Build index
01:21 286Mb   100.0% Searching finalQC_usearchfmt.06.fasta, 72.9% matched  80.0% Searching finalQC_usearchfmt.06.fasta, 73.3% matched
usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu



Processing /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.09.fasta


00:00 49Mb    100.0% Reading otusn.pick.fasta
00:00 15Mb    100.0% Masking (fastnucleo)    
00:00 16Mb    100.0% Word stats          
00:00 16Mb    100.0% Alloc rows
00:00 45Mb    100.0% Build index
01:20 285Mb   100.0% Searching finalQC_usearchfmt.09.fasta, 72.9% matched


...

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 49Mb    100.0% Reading otusn.pick.fasta
00:01 15Mb    100.0% Masking (fastnucleo)    
00:01 16Mb    100.0% Word stats          
00:01 16Mb    100.0% Alloc rows
00:01 45Mb    100.0% Build index
01:27 286Mb   100.0% Searching finalQC_usearchfmt.05.fasta, 72.9% matched


Processing /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.00.fasta


usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 49Mb    100.0% Reading otusn.pick.fasta
00:00 15Mb    100.0% Masking (fastnucleo)    
00:01 16Mb    100.0% Word stats          
00:01 16Mb    100.0% Alloc rows
00:01 45Mb    100.0% Build index
01:26 285Mb   100.0% Searching finalQC_usearchfmt.00.fasta, 72.9% matched 56.8% Searching finalQC_usearchfmt.00.fasta, 73.0% matched


Processing /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.07.fasta


usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 49Mb    100.0% Reading otusn.pick.fasta
00:00 15Mb    100.0% Masking (fastnucleo)    
00:00 16Mb    100.0% Word stats          
00:00 16Mb    100.0% Alloc rows
00:01 45Mb    100.0% Build index
01:33 285Mb   100.0% Searching finalQC_usearchfmt.07.fasta, 72.9% matched


In [74]:
ucFile = os.path.join(workDir, 'readmap.uc')
g = os.path.join(workDir, 'readmap0*.uc')
fileList = glob.glob(g)

filestring = ' '.join(fileList)

!cat $filestring > $ucFile

In [75]:
otu_table_file = 'otu_table.txt'
cmd = 'python2 /opt/edgar_python_scripts/uc2otutab.py {} > {}'.format(ucFile, otu_table_file)
!$cmd 

/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/readmap.uc 100.0%   


In [76]:
# make sure no existing biome file
otu_biom_file = 'otu_table.biom'
if os.path.isfile(otu_biom_file):
    os.unlink(otu_biom_file)
    
cmd = 'biom convert -i {} -o {} --table-type "OTU table" --to-json'.format(otu_table_file, otu_biom_file)
!$cmd

In [77]:
# make sure no existing biome file
otu_sum_file = 'otu_table_summary.txt'
if os.path.isfile(otu_sum_file):
    os.unlink(otu_sum_file)
    
cmd = 'biom summarize-table -i {} -o {}'.format(otu_biom_file, otu_sum_file)
!$cmd

In [5]:
otu_sum_file = 'otu_table_summary.txt'
!head -n 20 $otu_sum_file 

Num samples: 2295
Num observations: 30808
Total count: 52388415
Table density (fraction of non-zero values): 0.059

Counts/sample summary:
 Min: 10.0
 Max: 195083.0
 Median: 14865.000
 Mean: 22827.196
 Std. dev.: 24743.048
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
MR.M.13C-Xyl.D3.R1_Frac18_bad: 10.0
MR.A.13C-Van.D30.R3_Frac16_bad: 11.0
MR.A.13C-Van.D30.R3_Frac15_bad: 13.0
MR.A.13C-Van.D30.R3_Frac14_bad: 15.0
MR.F.13C-Van.D14.R2_Frac9: 21.0


## Adding taxonomy

In [7]:
# make sure no existing biome file
otu_biom_file = 'otu_table.biom'
otu_biom_tax_file = 'otu_table_wtax.biom'
otuTax = os.path.join(workDir, 'otusn_tax')
otuTax = os.path.join(otuTax, 'otusn_tax_assignments.txt')

if os.path.isfile(otu_biom_tax_file):
    os.unlink(otu_biom_tax_file)
    
# bio add-metadata
cmd = """biom add-metadata -i {} -o {} \
    --observation-metadata-fp {} \
    --sc-separated taxonomy \
    --float-fields consensus \
    --int-fields numhits \
    --observation-header OTUID,taxonomy,consensus,numhits"""
cmd = cmd.format(otu_biom_file, otu_biom_tax_file, otuTax)
!$cmd

In [80]:
!ls -thlc $otu_biom_tax_file

-rw-rw-r-- 1 sam sam 30M May 18 10:00 otu_table_wtax.biom


# Final files

In [81]:
!tail -n +2 $otu_table_file | wc -l | perl -pe 's/ .+//'

30808


In [82]:
print ('OTU table: {}'.format(otu_table_file))
print ('OTU pick file: {}'.format(otuFilePick))
print ('uclust formatted file: {}'.format(seqFile))

OTU table: otu_table.txt
OTU pick file: otusn.pick.fasta
uclust formatted file: /home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/OTU_binning/finalQC_usearchfmt.fasta


In [83]:
# Number of OTUs
!tail -n +2 $otu_table_file | wc -l | perl -pe 's/ .+//'
!grep -c ">" $otuFilePick
!grep -c ">" $seqFile

30808
30810
71891874


In [84]:
# total number of samples
sys.stdout.write('Total number of samples: ')
!head -n 1 $otu_table_file | perl -pe 's/[^\t]+\t//; s/\t/\n/g' | wc -l

Total number of samples: 2295
